![](https://i.imgur.com/A3Cxp7P.png)

## FIFA official API

https://api.fifa.com

### API Doc

https://api.qa.fifa.com/Help

>Our goal is to fetch world cup 2018 data through this API.

In [1]:
import csv
import json
import requests
from tqdm import tqdm

In [2]:
def write_to_csv(data, filename, fieldnames):
    with open(filename, "w") as f:
        csv_dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
        csv_dict_writer.writeheader()
        csv_dict_writer.writerows(data)

## Get list of available competitions

https://api.fifa.com/api/v1/competitions/search?name=world%20cup

In [3]:
def get_competitions(query):
    url = "https://api.fifa.com/api/v1/competitions/search"
    params = {
        "name": query
    }
    r = requests.get(url, params=params)
    data = r.json()['Results']
    return data

In [4]:
competitions = get_competitions("world cup")

In [5]:
competitions[0]

{'IdCompetition': '17',
 'Name': [{'Locale': 'en-GB', 'Description': 'FIFA World Cup™'}],
 'IdConfederation': [],
 'IdMemberAssociation': [],
 'IdOwner': 'FIFA',
 'Gender': 1,
 'FootballType': 0,
 'TeamType': 1,
 'CompetitionType': 3,
 'Properties': {'IdCBS': '17', 'IdIFES': 'FWC'},
 'IsUpdateable': None}

In [6]:
world_cup_id = competitions[0]['IdCompetition']

## Get list of all world cup seasons

https://api.fifa.com/api/v1/seasons?idCompetition=17

In [7]:
def get_seasons(competition_id):
    url = "https://api.fifa.com/api/v1/seasons"
    params = {
        "idCompetition": competition_id
    }
    r = requests.get(url, params=params)
    data = r.json()['Results']
    return data

In [8]:
seasons = get_seasons(world_cup_id)

In [9]:
len(seasons)

22

In [10]:
season_id = seasons[1]['IdSeason']

## Get stages

https://api.fifa.com/api/v1/stages?idCompetition=17&idSeason=254645

In [11]:
def get_stages(season_id, competition_id):
    url = "https://api.fifa.com/api/v1/stages"
    params = {
        "idCompetition": competition_id,
        "idSeason": season_id
    }
    r = requests.get(url, params=params)
    return r.json()['Results']

In [12]:
stages = get_stages(season_id, world_cup_id)

In [13]:
stages_map = {stage['IdStage']: stage['Name'][0]['Description'] for stage in stages}

## Get list of all world cup matches in a season

https://api.fifa.com/api/v1/calendar/matches?idseason=254645&idcompetition=17

In [14]:
def get_matches(competition_id, season_id):
    url = "https://api.fifa.com/api/v1/calendar/matches"
    params = {
        "idcompetition": competition_id,
        "idseason": season_id,
        "count": 100
    }
    r = requests.get(url, params=params)
    data = r.json()['Results']
    return data

In [15]:
world_cup_matches = get_matches(world_cup_id, season_id)

In [16]:
len(world_cup_matches)

64

## Generate CSV file for matches data

In [17]:
direct_fields = ['IdMatch', 'Attendance', 'MatchDay', 'Date', 'HomeTeamScore', 'AwayTeamScore', 'HomeTeamPenaltyScore',
                 'AwayTeamPenaltyScore', 'IsHomeMatch', 'MatchStatus', 'ResultType']

In [18]:
matches_data = []

In [19]:
for match in world_cup_matches:
    match_data = {}
    
    # set direct fields
    for field in direct_fields:
        match_data[field] = match[field]
    
    # set indirect fields
    match_data['Stage'] = stages_map[match['IdStage']]
    match_data['HomeTeamName'] = match['Home']['TeamName'][0]['Description']
    match_data['AwayTeamName'] = match['Away']['TeamName'][0]['Description']
    match_data['HomeTeamTactics'] = match['Home']['Tactics']
    match_data['AwayTeamTactics'] = match['Away']['Tactics']
    match_data['StadiumName'] = match['Stadium']['Name'][0]['Description']
    
    # set weather details
    match_data['Temperature'] = match['Weather']['Temperature']
    match_data['Humidity'] = match['Weather']['Humidity']
    match_data['WindSpeed'] = match['Weather']['WindSpeed']
    match_data['WeatherType'] = match['Weather']['TypeLocalized'][0]['Description']
        
    # set ball possession
    match_data['BallPossessionHome'] = match['BallPossession']['OverallHome']
    match_data['BallPossessionAway'] = match['BallPossession']['OverallAway']
    
    # set winner team name
    if match['Home']['IdTeam'] == match['Winner']:
        match_data['Winner'] = match_data['HomeTeamName']
    elif match['Away']['IdTeam'] == match['Winner']:
        match_data['Winner'] = match_data['AwayTeamName']
    else:
        match_data['Winner'] = None
    
    matches_data.append(match_data)

In [20]:
matches_data_headers = ['IdMatch', 'HomeTeamName', 'AwayTeamName', 'Attendance', 'MatchDay', 'Date', 'Stage', 
                        'HomeTeamTactics', 'AwayTeamTactics', 'HomeTeamScore', 'AwayTeamScore', 
                        'HomeTeamPenaltyScore', 'AwayTeamPenaltyScore', 'IsHomeMatch', 'MatchStatus', 
                        'ResultType', 'StadiumName', 'Temperature', 'Humidity', 'WindSpeed', 'WeatherType', 
                        'BallPossessionHome', 'BallPossessionAway', 'Winner']
write_to_csv(matches_data, "matches.csv", matches_data_headers)

## Get match details

https://api.fifa.com/api/v1/live/football/17/254645/275093/300331537

In [21]:
def get_match_detail(match_id, stage_id, season_id, competition_id):
    url = "https://api.fifa.com/api/v1/live/football/{}/{}/{}/{}".format(competition_id, season_id, stage_id, match_id)
    r = requests.get(url)
    data = r.json()
    return data

In [22]:
world_cup_matches_detailed = []

In [23]:
for match in tqdm(world_cup_matches):
    data = get_match_detail(match['IdMatch'], match['IdStage'], match['IdSeason'], match['IdCompetition'])
    world_cup_matches_detailed.append(data)

100%|██████████| 64/64 [01:17<00:00,  1.21s/it]


In [24]:
# Rememer this game?
world_cup_matches_detailed[3]['HomeTeam']['Goals']

[{'Type': 1,
  'IdPlayer': '201200',
  'Minute': "4'",
  'IdAssistPlayer': '269859',
  'Period': 3,
  'IdGoal': None,
  'IdTeam': '43963'},
 {'Type': 2,
  'IdPlayer': '201200',
  'Minute': "44'",
  'IdAssistPlayer': '269859',
  'Period': 3,
  'IdGoal': None,
  'IdTeam': '43963'},
 {'Type': 2,
  'IdPlayer': '201200',
  'Minute': "88'",
  'IdAssistPlayer': '269859',
  'Period': 5,
  'IdGoal': None,
  'IdTeam': '43963'}]

## Generate CSV file for goals data

In [25]:
goals_data = []

In [26]:
team_types = ['HomeTeam', 'AwayTeam']

for match in world_cup_matches_detailed:
    match_id = match['IdMatch']
    player_map = {player['IdPlayer']: player for player in match['HomeTeam']['Players']+match['AwayTeam']['Players']}
    for idx, team in enumerate(team_types):
        team_name = match[team]['TeamName'][0]['Description']
        other_team_name = match[team_types[(idx+1)%2]]['TeamName'][0]['Description']
        for goal in match[team]['Goals']:
            goal_data = {}
            goal_data['IdMatch'] = match_id
            goal_data['Type'] = goal['Type']
            goal_data['Minute'] = goal['Minute']
            goal_data['TeamName'] = team_name
            goal_data['OppositionTeamName'] = other_team_name
            goal_data['PlayerName'] = player_map[goal['IdPlayer']]['PlayerName'][0]['Description']
            goal_data['Position'] = player_map[goal['IdPlayer']]['Position']
            goal_data['PlayerShirtNumber'] = player_map[goal['IdPlayer']]['ShirtNumber']
            goal_data['IsCaptain'] = player_map[goal['IdPlayer']]['Captain']
            if goal['IdAssistPlayer']:
                goal_data['GoalKeeperName'] = player_map[goal['IdAssistPlayer']]['PlayerName'][0]['Description']
            else:
                goal_data['GoalKeeperName'] = None
            goals_data.append(goal_data)

In [27]:
# number of proper goals
len([goal['Minute'] for goal in goals_data if "120" not in goal['Minute']])

169

In [28]:
goals_data_headers = ['PlayerName', 'Minute', 'IdMatch', 'Type', 'TeamName', 'OppositionTeamName',  'Position', 'IsCaptain', 'GoalKeeperName', 'PlayerShirtNumber']
write_to_csv(goals_data, "goals.csv", goals_data_headers)

## Get squads

https://api.fifa.com/api/v1/teams/squads/all/17/254645?count=1000

In [29]:
def get_squads(competition_id, season_id):
    url = "https://api.fifa.com/api/v1/teams/squads/all/{}/{}".format(competition_id, season_id)
    params = {
        "count": 1000
    }
    r =requests.get(url)
    return r.json()['Results']

In [30]:
squads = get_squads(world_cup_id, season_id)

## Generate CSV for teams

In [31]:
def get_countries():
    url = "https://api.fifa.com/api/v1/countries"
    params = {
        "count": 1000
    }
    r = requests.get(url, params=params)
    return r.json()['Results']

In [32]:
countries = get_countries()
country_map = {country['IdCountry']: country['Name'] for country in countries}

In [33]:
teams_data = []

In [34]:
for squad in squads:
    team_data = {}
    team_data['IdTeam'] = squad['IdTeam']
    team_data['TeamName'] = squad['TeamName'][0]['Description']
    team_data['CoachName'] = squad['Officials'][0]['Name'][0]['Description']
    team_data['CoachCountry'] = country_map[squad['Officials'][0]['IdCountry']]
    teams_data.append(team_data)

In [35]:
teams_data_headers = ['IdTeam', 'TeamName', 'CoachName', 'CoachCountry']
write_to_csv(teams_data, "teams.csv", teams_data_headers)

## Get players data

https://api.fifa.com/api/v1/players/201200

In [36]:
def get_player_data(player_id):
    url = "https://api.fifa.com/api/v1/players/{}".format(player_id)
    r = requests.get(url)
    data = r.json()
    player_data = {}
    player_data['Name'] = data['Name'][0]['Description']
    for field in ['IdPlayer', 'BirthDate', 'Weight', 'Height', 'InternationalCaps', 'Goals']:
        player_data[field] = data[field]
    return player_data

## Generate CSV file for players data

In [37]:
players_data = []

In [38]:
for squad in squads:
    team = squad['TeamName'][0]['Description']
    for player in tqdm(squad['Players'], desc=team):
        player_data = get_player_data(player['IdPlayer'])
        player_data['TeamName'] = team
        player_data['Position'] = player['PositionLocalized'][0]['Description']
        players_data.append(player_data)

Belgium: 100%|██████████| 23/23 [00:27<00:00,  1.17s/it]


In [39]:
players_data_headers = ['IdPlayer', 'Name', 'TeamName', 'Position', 'BirthDate', 'Weight', 'Height', 'InternationalCaps', 'Goals']
write_to_csv(players_data, "players.csv", players_data_headers)